# Greedy simulated annealing

In [159]:
using Printf, DelimitedFiles
include("../code/headers.jl")

In [2]:
H_core = readgraph("/tmp/graph9000.txt")
H_nocore = readseeds("/tmp/seeds9000.txt", H_core)
m,n = size(H_core)
rate_core = 1 - m/n
M,N = size(H_nocore)
rate_nocore = 1 - M/N

lm_core = LossyModel(FactorGraphGF2(H_core))
lm_nocore = LossyModel(FactorGraphGF2(H_nocore));

size(H) = (6600, 9000)
5 removed factors [129, 1515, 1531, 1655, 3471]


**Strategy**:
1. With no external field, perform "greedy" moves by accepting only $\Delta E\le 0$
2. If a codeword is found in this way, start flipping spins that are not aligned with the external field. Accept if the system stays SAT

In [41]:
function greedy_simann_checks(lm::LossyModel, maxiter1::Int=Int(1e7); 
        idxperm=randperm(lm.fg.n), thresh=1000, maxlag=2)
    lm.beta1 = 1.0
    lm.beta2 = 0.0
    E = Int(energy(lm))
    E1_history = [E]
    iters1_history = [1] 
    niters1 = maxiter1
    
    for it in 2:maxiter1
        # Pick the new site in the permutation of 1,...,n
        to_flip = idxperm[mod1(it, lm.fg.n)]
        ΔE = energy_shift_checks(lm, to_flip)
        if ΔE <= 0
            lm.x[to_flip] = !lm.x[to_flip]
            E += ΔE
            if ΔE != 0
                push!(E1_history, E)
                push!(iters1_history,it)
            end
        else
            # Check if the last energy update was more than `1/maxlag` of a simulation ago and at least
            #  `thresh` iterations have been performed
            if (it/iters1_history[end] > maxlag && it>thresh) || E==0
                niters1 = it
                break
            end
        end
        mod(it, lm.fg.n)==0 && shuffle!(idxperm)
    end
    return E1_history, iters1_history, niters1
end

function greedy_simann_overlap(lm::LossyModel, maxiter2=Int(1e5), E_init=0)
    
    E2_history = [E_init]
    iters2_history = [1]
    niters2 = maxiter2
    not_aligned = (1:lm.fg.n)[lm.x .!= lm.y]
    for it in 2:maxiter2
        # propose spin to flip among those misaligned with the external field
        j = rand(1:length(not_aligned))
        to_flip = not_aligned[j]
        # accept only if the system stays SAT
        ΔE = energy_shift_checks(lm, to_flip)
        if ΔE <= 0
            lm.x[to_flip] = !lm.x[to_flip]
            push!(iters2_history, it)
            deleteat!(not_aligned, j)
        end
    end
    return iters2_history, niters2
end
    
# Compute shift in parity-check energy due to flipping one variable. NO EXT FIELDS INVOLVED!
function energy_shift_checks(lm::LossyModel, to_flip::Int)
    dE = 0
    for a in lm.fg.Vneigs[to_flip]
        z = 0
        for w in lm.fg.Fneigs[a]    
            z = xor(z, lm.x[w])    
        end
        dE += 1-2*z
    end
    return dE
end

# function greedy_simann(lm::LossyModel, maxiter1=Int(1e7), maxiter2=Int(1e5); 
#         idxperm=randperm(lm.fg.n), thresh=1000, maxlag=2)
    
#     # PART ONE: GO TO A CODEWORD  
#     E1_history, iters1_history, niters1 = greedy_simann_checks(lm, maxiter1, 
#         idxperm=idxperm, thresh=thresh, maxlag=maxlag)
    
#     # PART TWO: TRY TO ALIGN TO SOURCE. Only if a codeword was reached in part 1
#     codeword = (E1_history[end]==0)
#     if codeword
#         iters2_history, niters2 = greedy_simann_overlap(lm, maxiter2, 0)
#     end    
#     return E1_history, iters1_history, niters1, codeword, iters2_history, niters2
# end

Look for codewords and save them when found

In [66]:
function greedy_simann_checks_trials(lm::LossyModel, N::Int=Int(5e1), maxiter=Int(1e8), args...;
        maxlag=2, kw...)
    min_energies = zeros(Int,N)
    logstr = Vector{String}(undef,N)
    init = bitrand(lm.fg.n)
    X = BitArray{1}[]
    E1_history = [Int[] for i in 1:N]
    iters1_history = [Int[] for i in 1:N]
    distance_history = zeros(Int, N)
    @showprogress for i in 1:N
        init .= bitrand(lm.fg.n)
        lm.x = copy(init)
        E1_history[i], iters1_history[i], niters1 = greedy_simann_checks(lm, maxiter, maxlag=maxlag)
        distance_history[i] = hd(lm.x, init)
        min_energies[i] = E1_history[i][end]
        logstr[i] = string("Sim ", @sprintf("%3d", i) , " of $N:  E=", @sprintf("%4d", E1_history[i][end]), " after ",
            @sprintf("%8d",niters1), " iters.  dₕ=", distance_history[i])
        min_energies[i] != 0 && push!(X, copy(lm.x))    
    end
    return min_energies, logstr, X, E1_history, iters1_history, distance_history
end

greedy_simann_checks_trials (generic function with 3 methods)

On the found codewords, try moving towards the external field

In [36]:
function greedy_simann_overlap_trials(lm::LossyModel, Einit::Vector{Int}, X::Vector{BitArray{1}}, maxiter=Int(1e8))
    N = length(X)
    logstr = Vector{String}(undef,N)
    dist = zeros(N)
    @showprogress for i in eachindex(X)
        lm.x .= X[i]
        d = distortion(lm)
        iters2_history, niters2 = greedy_simann_overlap(lm, maxiter)
        dist[i] = distortion(lm)
        logstr[i] = string("Sim ", @sprintf("%3d", i) , " of $N.  Flips ", length(iters2_history)-1,
            ".  Distortion before and after ", @sprintf("%.3f %.3f",d,dist[i]))
    end
    return dist, logstr
end

greedy_simann_overlap_trials (generic function with 4 methods)

In [37]:
init = bitrand(n)
lm_core.x = copy(init)
E_history, iters_history, niters = greedy_simann_checks(lm_core, Int(1e8))

([3307, 3306, 3304, 3302, 3301, 3299, 3298, 3296, 3293, 3291  …  14, 12, 10, 9, 8, 6, 5, 4, 2, 0], [1, 5, 8, 9, 13, 15, 16, 17, 24, 25  …  2324515, 2725657, 3227750, 3256155, 5367903, 6338582, 6986573, 7578054, 13840187, 20800346], 20800347)

In [62]:
using BenchmarkTools
foo() = begin
    init = bitrand(n)
    lm_core.x = copy(init)
    E_history, iters_history, niters = greedy_simann_checks(lm_core, Int(1e8))
    nothing
end
    
@btime foo()

  1.031 s (8186 allocations: 267.75 KiB)


([3299, 3297, 3295, 3293, 3291, 3289, 3286, 3285, 3283, 3280  …  24, 22, 20, 18, 16, 14, 12, 10, 8, 7], [1, 13, 17, 20, 22, 25, 28, 31, 39, 40  …  964610, 1248240, 1376858, 1403658, 1513244, 1538025, 1579197, 1786682, 2598971, 2646696], 5293393)

## 1. Full core

In [ ]:
N = Int(5e2)
maxiter1 = Int(1e8);
min_energies_core, logstr1_core, X_core, E1_history_core, iters1_history_core, distance_history_core = 
    greedy_simann_checks_trials(lm_core, N, maxiter1);
sort(countmap(min_energies_core))

In [ ]:
logstr1_core

In [ ]:
maxiter = Int(1e8)
dist_core, logstr2_core = greedy_simann_overlap_trials(lm_core, min_energies_core[min_energies_core.!=0], 
    X_core, maxiter)

In [ ]:
logstr2_core

Compute the distribution of times to get to energy E

In [169]:
E_core = [0,1,2,4,8,16,32, 128, 264]
tau_core = [Int[] for e in E_core]
# Save the time at which each instance got at energy E
for (e,E) in enumerate(E_core)
    for (j,t) in enumerate(min_energies_core)
        # Find the index in vector of energy history corresponding to E
        idx = findfirst(<=(E), E1_history_core[j])
        idx !== nothing && push!(tau_core[e], iters1_history_core[j][idx]) 
    end
#     sort!(tau[e])
end

In [170]:
plts_core = Vector{Plots.Plot}(undef, length(E_core))
for e in eachindex(E_core)
    ydata = (1:length(tau_core[e])) ./ N
   plts_core[e] = Plots.scatter(sort(tau_core[e]), ydata, label="E=$(E_core[e])", legend=:topleft, ms=3, msw=0) 
    xlabel!("Iteration")
end

In [171]:
println("FULL CORE:")
# pl_core = plot(plts_core..., size=(800,800))
pl_core = plot(plts_core..., size=(1500,1000), dpi=200)
savefig(pl_core, "tauE_core.png")

FULL CORE:


In [177]:
open("files_montecarlo/full_core/energies.txt", "w") do io
    writedlm(io, E1_history_core, " ")
end
open("files_montecarlo/full_core/iters.txt", "w") do io
    writedlm(io, iters1_history_core, " ")
end
open("files_montecarlo/full_core/distances.txt", "w") do io
    writedlm(io, distance_history_core, " ")
end
open("files_montecarlo/full_core/distortions.txt", "w") do io
    writedlm(io, dist_core, " ")
end
open("files_montecarlo/full_core/tauE.txt", "w") do io
    writedlm(io, tau_core, " ")
end
open("files_montecarlo/full_core/full_log1.txt", "w") do io
    writedlm(io, logstr1_core, "\n")
end
open("files_montecarlo/full_core/full_log2.txt", "w") do io
    writedlm(io, logstr2_core, "\n")
end

## 2. Empty core

In [67]:
N = Int(5e2)
maxiter1 = Int(1e8);
min_energies_nocore, logstr1_nocore, X_nocore, E1_history_nocore, iters1_history_nocore, distance_history_nocore = 
    greedy_simann_checks_trials(lm_nocore, N, maxiter1);
sort(countmap(min_energies_nocore))

Progress: 100%|█████████████████████████████████████████| Time: 0:25:19


OrderedCollections.OrderedDict{Int64,Int64} with 11 entries:
  0  => 70
  1  => 126
  2  => 115
  3  => 83
  4  => 53
  5  => 26
  6  => 13
  7  => 8
  8  => 2
  9  => 2
  12 => 2

In [74]:
hist_energies_nocore = sort(countmap(min_energies_nocore))

OrderedCollections.OrderedDict{Int64,Int64} with 11 entries:
  0  => 70
  1  => 126
  2  => 115
  3  => 83
  4  => 53
  5  => 26
  6  => 13
  7  => 8
  8  => 2
  9  => 2
  12 => 2

In [68]:
logstr1_nocore

500-element Array{String,1}:
 "Sim   1 of 500:  E=   2 after 15955713 iters.  dₕ=4052"
 "Sim   2 of 500:  E=   0 after  7564071 iters.  dₕ=4123"
 "Sim   3 of 500:  E=   3 after 34360689 iters.  dₕ=4133"
 "Sim   4 of 500:  E=   3 after 29990881 iters.  dₕ=4068"
 "Sim   5 of 500:  E=   2 after 28070299 iters.  dₕ=4123"
 "Sim   6 of 500:  E=   5 after 14633513 iters.  dₕ=4221"
 "Sim   7 of 500:  E=   2 after 39852739 iters.  dₕ=4125"
 "Sim   8 of 500:  E=   3 after 10819953 iters.  dₕ=4088"
 "Sim   9 of 500:  E=   3 after 21181587 iters.  dₕ=4068"
 "Sim  10 of 500:  E=   1 after 18873029 iters.  dₕ=4176"
 "Sim  11 of 500:  E=   3 after 11348809 iters.  dₕ=4126"
 "Sim  12 of 500:  E=   2 after 15360589 iters.  dₕ=4052"
 "Sim  13 of 500:  E=   3 after 11210747 iters.  dₕ=3960"
 ⋮
 "Sim 489 of 500:  E=   8 after  3901263 iters.  dₕ=4104"
 "Sim 490 of 500:  E=   3 after 11880825 iters.  dₕ=4076"
 "Sim 491 of 500:  E=   2 after 14465805 iters.  dₕ=4155"
 "Sim 492 of 500:  E=   2 after 23435431

In [69]:
maxiter = Int(1e8)
dist_nocore, logstr2_nocore = greedy_simann_overlap_trials(lm_nocore, min_energies_nocore[min_energies_nocore.!=0], 
    X_nocore, maxiter)
logstr2_nocore

Progress: 100%|█████████████████████████████████████████| Time: 1:34:13


430-element Array{String,1}:
 "Sim   1 of 430.  Flips 1.  Distortion before and after 0.503 0.503"
 "Sim   2 of 430.  Flips 1.  Distortion before and after 0.488 0.488"
 "Sim   3 of 430.  Flips 0.  Distortion before and after 0.497 0.497"
 "Sim   4 of 430.  Flips 2.  Distortion before and after 0.500 0.500"
 "Sim   5 of 430.  Flips 1.  Distortion before and after 0.505 0.504"
 "Sim   6 of 430.  Flips 0.  Distortion before and after 0.494 0.494"
 "Sim   7 of 430.  Flips 0.  Distortion before and after 0.487 0.487"
 "Sim   8 of 430.  Flips 1.  Distortion before and after 0.499 0.499"
 "Sim   9 of 430.  Flips 0.  Distortion before and after 0.498 0.498"
 "Sim  10 of 430.  Flips 1.  Distortion before and after 0.496 0.495"
 "Sim  11 of 430.  Flips 2.  Distortion before and after 0.489 0.489"
 "Sim  12 of 430.  Flips 2.  Distortion before and after 0.499 0.499"
 "Sim  13 of 430.  Flips 17.  Distortion before and after 0.501 0.499"
 ⋮
 "Sim 419 of 430.  Flips 16.  Distortion before and after

In [155]:
E_nocore = [0,1,2,4,8,16,32, 128, 264]
tau_nocore = [Int[] for e in E_nocore]
# Save the time at which each instance got at energy E
for (e,E) in enumerate(E_nocore)
    for (j,t) in enumerate(min_energies_nocore)
        # Find the index in vector of energy history corresponding to E
        idx = findfirst(<=(E), E1_history_nocore[j])
        idx !== nothing && push!(tau_nocore[e], iters1_history_nocore[j][idx]) 
    end
end

In [156]:
plts_nocore = Vector{Plots.Plot}(undef, length(E_nocore))
for e in eachindex(E_nocore)
    ydata = (1:length(tau_nocore[e])) ./ N
   plts_nocore[e] = Plots.scatter(sort(tau_nocore[e]), ydata, label="E=$(E_nocore[e])", legend=:topleft, ms=3, msw=0) 
    xlabel!("Iteration")
end

In [158]:
println("EMPTY CORE:")
# pl_core = plot(plts_nocore..., size=(800,800))
pl_nocore = plot(plts_nocore..., size=(1500,1000), dpi=200)
savefig(pl_nocore, "tauE_nocore.png")

EMPTY CORE:


In [178]:
open("files_montecarlo/empty_core/energies.txt", "w") do io
    writedlm(io, E1_history_nocore, " ")
end
open("files_montecarlo/empty_core/iters.txt", "w") do io
    writedlm(io, iters1_history_nocore, " ")
end
open("files_montecarlo/empty_core/distances.txt", "w") do io
    writedlm(io, distance_history_nocore, " ")
end
open("files_montecarlo/empty_core/distortions.txt", "w") do io
    writedlm(io, dist_nocore, " ")
end
open("files_montecarlo/empty_core/tauE.txt", "w") do io
    writedlm(io, tau_nocore, " ")
end
open("files_montecarlo/empty_core/full_log1.txt", "w") do io
    writedlm(io, logstr1_nocore, "\n")
end
open("files_montecarlo/empty_core/full_log2.txt", "w") do io
    writedlm(io, logstr2_nocore, "\n")
end